App for scrapping data of movies budget and gross from the website https://www.the-numbers.com/movie/budgets/all

In [2]:
!pip3 install requests
!pip3 install beatifulsoup4
!pip3 install pandas

import requests
from bs4 import BeautifulSoup
import pandas as pd

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached requests-2.28.0-py3-none-any.whl (62 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached certifi-2022.5.18.1-py3-none-any.whl (155 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, plea

In [3]:
# Custom iterator for pages

class PageIter:
    def __iter__(self):
        self.a = 1
        return self

    def __next__(self):
        x = self.a
        self.a += 100
        return x

In [4]:
# Scraping function

def scrape(limit=None):
    all_data = {}
    for page in iter(PageIter()):
        resp = get_per_page(page)
        if resp is None or page > limit:
            break
        all_data.update(resp)
    load_to_pandas(all_data)

In [5]:
# call to specific page

def get_per_page(page=1):
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    }
    r = requests.get(f'https://www.the-numbers.com/movie/budgets/all/{page}', headers=headers)
    print(r.status_code, 'on', page, 'entries')
    if r.status_code != 200:
        return None
    return search_data(r)

In [6]:
# search data on given response

def search_data(resp):
    soup = BeautifulSoup(resp.content, 'html.parser')
    table = soup.find_all('table')
    table_data = {}
    if table:
        for row in table[0].find_all('tr'):
            cols = row.find_all('td')
            if cols:
                table_data[cols[2].text] = {
                    'release_date': cols[1].text,
                    'production_budget': cols[3].text,
                    'domestic_gross': cols[4].text,
                    'worldwide_gross': cols[5].text,
                }
    return table_data

In [7]:
# load data to pandas and output

def load_to_pandas(data):
    df = pd.DataFrame.from_dict(data)
    print(df)

In [8]:
# function call with entries limit (can be empty)

scrape(1000)

200 on 1 entries
200 on 101 entries
200 on 201 entries
200 on 301 entries
200 on 401 entries
200 on 501 entries
200 on 601 entries
200 on 701 entries
200 on 801 entries
200 on 901 entries
200 on 1001 entries
                  Avengers: Endgame  \
release_date           Apr 23, 2019   
production_budget      $400,000,000   
domestic_gross         $858,373,000   
worldwide_gross      $2,797,800,564   

                  Pirates of the Caribbean: On Stranger Tides  \
release_date                                     May 20, 2011   
production_budget                                $379,000,000   
domestic_gross                                   $241,071,802   
worldwide_gross                                $1,045,713,802   

                  Avengers: Age of Ultron  \
release_date                 Apr 22, 2015   
production_budget            $365,000,000   
domestic_gross               $459,005,868   
worldwide_gross            $1,395,316,979   

                  Star Wars Ep. VII: The For